
IMPORTS:

In [0]:
import pickle
import os
from os import path

import matplotlib.pyplot as plt

import numpy as np

import sys
import torch

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import cv2
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Model
from PIL import Image
import string
from keras.callbacks import ModelCheckpoint

Download data from drive to speed things up

In [0]:
zip_path = '/content/drive/My Drive/KNN/2017-IWT4S-CarsReId_LP-dataset.zip'
!cp "{zip_path}" .
!unzip -q '2017-IWT4S-CarsReId_LP-dataset.zip'

Parse csv and load train/test data

In [0]:
train_images = []
train_labels = []
test_images  = []
test_labels  = []

with open('trainVal.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            # Skip header row
            line_count += 1
        else:
            if int(row[3]) == 0:
                if(len(row[2]) != 7):
                    continue
                img = cv2.imread(row[1])
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                res = cv2.resize(img, dsize=(200, 40), interpolation=cv2.INTER_CUBIC)
                normalizedImg = np.zeros((200, 40))
                normalizedImg = cv2.normalize(res,  normalizedImg, 0, 255, cv2.NORM_MINMAX)
                test_labels.append([row[2]])
                test_images.append(normalizedImg)
            else:
                if(len(row[2]) != 7):
                    continue
                img = cv2.imread(row[1])
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                res = cv2.resize(img, dsize=(200, 40), interpolation=cv2.INTER_CUBIC)
                normalizedImg = np.zeros((200, 40))
                normalizedImg = cv2.normalize(res,  normalizedImg, 0, 255, cv2.NORM_MINMAX)
                train_labels.append([row[2]])
                train_images.append(normalizedImg)

train_images = np.asarray(train_images)
train_labels = np.asarray(train_labels)
test_images  = np.asarray(test_images)
test_labels  = np.asarray(test_labels)

# Debug
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

Convert to 'loss function friendly' format

In [0]:
# Create dict to encode digits to numbers
y_map = {}
for i,d in enumerate(string.digits+string.ascii_uppercase):
    y_map[d] = i

# Utility function for split string in char and encode them
def split_encode(x):
    x = list(x[0])
    x = [y_map[d] for d in x]
    return x

# Transform target
train_labels = np.apply_along_axis(split_encode, 1, train_labels)
test_labels = np.apply_along_axis(split_encode, 1, test_labels)

# Debug
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

Class names A-Z, 0-9

In [0]:
chars  = [chr(i) for i in list(range(ord('A'),ord('Z')+1))]
digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

class_names = digits + chars

print(class_names)
print(len(class_names))

Check few images

In [0]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

Create model

In [0]:
# 32,64,128 - filters count
# (x, x) is size of convolutional kernel
# relu is activation function
# batch normalization after each convolutional layer

from keras.layers import *

inp_dim = (40, 200, 3)
x_inp = Input(shape=inp_dim)

# First part
x = Conv2D(32, (3, 3), use_bias=False, padding='same')(x_inp)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(32, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(32, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((2, 2))(x)

# Second part
x = Conv2D(64, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(64, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(64, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((2, 2))(x)

# Third part
x = Conv2D(128, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(128, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(128, (3, 3), use_bias=False, padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((2, 2))(x)


x_out = Flatten()(x)
model = Model(x_inp, x_out)

branches = []

for _ in range(7):
    x = Dense(128)(x_out)
    x = Dense(36)(x)
    x = Activation("softmax")(x)
    branches.append(x)

final_model = Model(inputs = x_inp, outputs = branches)

# Display model architecture
final_model.summary()

Compile and train the model

In [0]:
# Checkpoint callback (to save weights after each epoch)
filepath = "/content/drive/My Drive/KNN/model-big-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1)

# Adam optimizer with 0.001 learning rate
final_model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['accuracy'])

# Train for 80 epochs
history = final_model.fit(train_images, [train_labels[:,i] for i in range(7)], epochs=80,
                          validation_data=(test_images, [test_labels[:,i] for i in range(7)]), callbacks=[checkpoint])

Evaluate the model

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print(test_acc)